In [1]:
# Extract Features from data set - DS 1: MER Audio Traffic dataset: Consider 1 mp3 file as one observation 
# - without Hilbert Transform
import os
os.chdir("D:/PhD Program/Final Research/OLR")

import pandas as pd
import Feature_Extraction_Util_V2 as util
import os

num_mfcc = 20
num_mel = 64
mel_cutoff_val = 0.01
hilbert_transform_val = "N"
MFCC_flag = "Y" 
feature_file_name = "DS_1_Feature"

if MFCC_flag == "Y":
    feature_file_name = feature_file_name + "_MFCC"
if hilbert_transform_val == "Y":
    feature_file_name = feature_file_name + "_hilbert_trans"
else:
    feature_file_name = feature_file_name + "_no_hilbert_trans"

feature_file_name = feature_file_name + ".csv"
print(feature_file_name)

feat_util = util.Feature_Util()
root_path = "D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset/"
class_names = ["Q1","Q2","Q3","Q4"]
#class_names = ["Q1"]
file_cntr = 1
for class_name in class_names:
    data_path = root_path+"/"+class_name
    for root, dirs, files in os.walk(data_path):
        for file_name in files:

            file_path = data_path +"/" +file_name
            db_row = {'sound_file_name': [file_name],
                      'sound_file_class': [class_name]}
            # read sound file
            ret_val, msg_val, signal_val, sr = feat_util.readSignal(file_path,hilbert_transform=hilbert_transform_val)
            # get features and aggregations
            # F0
            ret_val, msg_str, feature_val =  feat_util.getFZero(signal_val)
            agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="F0",mean_agg = "Y", median_agg = "Y", max_agg = "Y", min_agg = "Y", IQR_agg = "Y", sd_agg ="N")
            db_row = db_row | agg_val

            # RMS
            ret_val,msg_str, feature_val =  feat_util.getRMS(signal_val)
            agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="RMS",mean_agg = "Y", median_agg = "Y", max_agg = "Y", min_agg = "N", IQR_agg = "Y", sd_agg ="N")
            db_row = db_row | agg_val

            # Spectral Centroid
            ret_val,msg_str, feature_val =  feat_util.getSpectralCentroid(signal_val)
            agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="spect_centroid")
            db_row = db_row | agg_val

            # Spectral Roll Off
            ret_val,msg_str, feature_val =  feat_util.getSpectralRolloff(signal_val)
            agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="spect_rolloff")
            db_row = db_row | agg_val

            # Spectral Flatness
            ret_val,msg_str, feature_val =  feat_util.getSpectralFlatness(signal_val)
            agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="spect_flat")
            db_row = db_row | agg_val

            # Spectral BW
            ret_val,msg_str, feature_val =  feat_util.getSpectralBW(signal_val)
            agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="spect_bw")
            db_row = db_row | agg_val

            # Spectral Contrast
            ret_val,msg_str, feature_val =  feat_util.getSpectralContrast(signal_val)
            agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="spect_contrast")
            db_row = db_row | agg_val

            # ZCR
            ret_val,msg_str, feature_val =  feat_util.getZCR(signal_val)
            agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="ZCR")
            db_row = db_row | agg_val

            # Beat / Tempo
            ret_val,msg_str, feature_val =  feat_util.getTempo(signal_val, sr)
            agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="tempo")
            db_row = db_row | agg_val

            # PLP
            ret_val,msg_str, feature_val =  feat_util.getPLP(signal_val, sr)
            agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="plp")
            db_row = db_row | agg_val

            # Power
            ret_val,msg_str, feature_val =  feat_util.getPower(signal_val, sr=sr)
            agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="power")
            db_row = db_row | agg_val

            # get MFCC's - to be included later
            if MFCC_flag == "Y":
                for mfcc_count in range(num_mfcc):
                    mfcc_str = "MFCC_" + str(mfcc_count)
                    ret_val, msg_str, feature_val = feat_util.getMFCC(signal_val, sr=sr, pos_val=mfcc_count)
                    agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name=mfcc_str)
                    db_row = db_row | agg_val

            # get MEL
            for mel_count in range(num_mel):
                mel_str = "mel_" + str(mel_count)
                ret_val, msg_str, feature_val = feat_util.getMel(signal_val, sr=sr, pos_val=mel_count, cutoff_val=mel_cutoff_val)
                agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name=mel_str)
                db_row = db_row | agg_val

            # get Loudness
            ret_val,msg_str, feature_val =  feat_util.getLoudness(file_path)
            #agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="loudness")
            db_row = db_row | {'loudness_mean':[feature_val]}

            # get Chromagram - To be included later. Hence commenting.
            # ret_val,msg_str, feature_val = feat_util.getChromagram(signal_val, sr=sr)
            # agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="chromagram")
            # db_row = db_row | agg_val


            data_df = pd.DataFrame(db_row)

            # Build feature dataset
            if file_cntr == 1:
                final_df = data_df
                #break
            else:
                final_df = pd.concat([final_df,data_df],axis=0)
            file_cntr += 1
            print("Extracted feature from file: "+str(file_path))

# Save feature dataset
final_df = final_df.reset_index()
final_df = final_df.drop(columns = ['index'])
final_df.to_csv(feature_file_name, index=False)

DS_1_Feature_MFCC_no_hilbert_trans.csv


D:\MyApps\anaconda3\lib\site-packages\librosa\core\audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0000040632.mp3


D:\MyApps\anaconda3\lib\site-packages\librosa\core\audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0000082187.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0000249842.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0000364027.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0000392975.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0000414517.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0000442827.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0000540286.mp3
Extracted feature from file: D:/PhD Program/Final Research/DAT

...

D:\MyApps\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
D:\MyApps\anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0000746822.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0000762097.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0000821772.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0000888329.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0000901959.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0000922312.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0000980148.mp3
Extracted feature from file: D:/PhD Program/Final Research/DAT

Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0005798527.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0005850999.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0005884980.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0005987148.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0006037085.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0006107326.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0006117018.mp3
Extracted feature from file: D:/PhD Program/Final Research/DAT

Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0012027536.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0012112600.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0012151268.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0012222183.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0012333553.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0012383717.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0012766840.mp3
Extracted feature from file: D:/PhD Program/Final Research/DAT

Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0030495725.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0031495046.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0031807708.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0031826774.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0031827474.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0031886830.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0031898123.mp3
Extracted feature from file: D:/PhD Program/Final Research/DAT

In [1]:
# Extract Features from data set - DS 1: MER Audio Traffic dataset: Consider 1 mp3 file as one observation 
# - with Hilbert Transform
import os
os.chdir("D:/PhD Program/Final Research/OLR")

import pandas as pd
import Feature_Extraction_Util_V2 as util
import os

num_mfcc = 20
num_mel = 64
mel_cutoff_val = 0.01
hilbert_transform_val = "Y"
MFCC_flag = "Y"  
feature_file_name = "DS_1_Feature"

if MFCC_flag == "Y":
    feature_file_name = feature_file_name + "_MFCC"
if hilbert_transform_val == "Y":
    feature_file_name = feature_file_name + "_hilbert_trans"
else:
    feature_file_name = feature_file_name + "_no_hilbert_trans"

feature_file_name = feature_file_name + ".csv"
print(feature_file_name)

feat_util = util.Feature_Util()
root_path = "D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset/"
class_names = ["Q1","Q2","Q3","Q4"]
#class_names = ["Q1"]
file_cntr = 1
for class_name in class_names:
    data_path = root_path+"/"+class_name
    for root, dirs, files in os.walk(data_path):
        for file_name in files:

            file_path = data_path +"/" +file_name
            db_row = {'sound_file_name': [file_name],
                      'sound_file_class': [class_name]}
            # read sound file
            ret_val, msg_val, signal_val, sr = feat_util.readSignal(file_path,hilbert_transform=hilbert_transform_val)
            # get features and aggregations
            # F0
            ret_val, msg_str, feature_val =  feat_util.getFZero(signal_val)
            agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="F0",mean_agg = "Y", median_agg = "Y", max_agg = "Y", min_agg = "Y", IQR_agg = "N", sd_agg ="N")
            db_row = db_row | agg_val

            # RMS
            ret_val,msg_str, feature_val =  feat_util.getRMS(signal_val)
            agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="RMS",mean_agg = "Y", median_agg = "Y", max_agg = "Y", min_agg = "N", IQR_agg = "N", sd_agg ="N")
            db_row = db_row | agg_val

            # Spectral Centroid
            ret_val,msg_str, feature_val =  feat_util.getSpectralCentroid(signal_val)
            agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="spect_centroid")
            db_row = db_row | agg_val

            # Spectral Roll Off
            ret_val,msg_str, feature_val =  feat_util.getSpectralRolloff(signal_val)
            agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="spect_rolloff")
            db_row = db_row | agg_val

            # Spectral Flatness
            ret_val,msg_str, feature_val =  feat_util.getSpectralFlatness(signal_val)
            agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="spect_flat")
            db_row = db_row | agg_val

            # Spectral BW
            ret_val,msg_str, feature_val =  feat_util.getSpectralBW(signal_val)
            agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="spect_bw")
            db_row = db_row | agg_val

            # Spectral Contrast
            ret_val,msg_str, feature_val =  feat_util.getSpectralContrast(signal_val)
            agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="spect_contrast")
            db_row = db_row | agg_val

            # ZCR
            ret_val,msg_str, feature_val =  feat_util.getZCR(signal_val)
            agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="ZCR")
            db_row = db_row | agg_val

            # Beat / Tempo
            ret_val,msg_str, feature_val =  feat_util.getTempo(signal_val, sr)
            agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="tempo")
            db_row = db_row | agg_val

            # PLP
            ret_val,msg_str, feature_val =  feat_util.getPLP(signal_val, sr)
            agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="plp")
            db_row = db_row | agg_val

            # Power
            ret_val,msg_str, feature_val =  feat_util.getPower(signal_val, sr=sr)
            agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="power")
            db_row = db_row | agg_val

            # get MFCC's - to be included later
            if MFCC_flag == "Y":
                for mfcc_count in range(num_mfcc):
                    mfcc_str = "MFCC_" + str(mfcc_count)
                    ret_val, msg_str, feature_val = feat_util.getMFCC(signal_val, sr=sr, pos_val=mfcc_count)
                    agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name=mfcc_str)
                    db_row = db_row | agg_val

            # get MEL
            for mel_count in range(num_mel):
                mel_str = "mel_" + str(mel_count)
                ret_val, msg_str, feature_val = feat_util.getMel(signal_val, sr=sr, pos_val=mel_count, cutoff_val=mel_cutoff_val)
                agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name=mel_str)
                db_row = db_row | agg_val

            # get Loudness
            ret_val,msg_str, feature_val =  feat_util.getLoudness(file_path)
            #agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="loudness")
            db_row = db_row | {'loudness_mean':[feature_val]}

            # get Chromagram - To be included later. Hence commenting.
            # ret_val,msg_str, feature_val = feat_util.getChromagram(signal_val, sr=sr)
            # agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="chromagram")
            # db_row = db_row | agg_val


            data_df = pd.DataFrame(db_row)

            # Build feature dataset
            if file_cntr == 1:
                final_df = data_df
                #break
            else:
                final_df = pd.concat([final_df,data_df],axis=0)
            file_cntr += 1
            print("Extracted feature from file: "+str(file_path))

# Save feature dataset
final_df = final_df.reset_index()
final_df = final_df.drop(columns = ['index'])
final_df.to_csv(feature_file_name, index=False)

DS_1_Feature_MFCC_hilbert_trans.csv


D:\MyApps\anaconda3\lib\site-packages\librosa\core\audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0000040632.mp3


D:\MyApps\anaconda3\lib\site-packages\librosa\core\audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0000082187.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0000249842.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0000364027.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0000392975.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0000414517.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0000442827.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0000540286.mp3
Extracted feature from file: D:/PhD Program/Final Research/DAT

...

Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0005091539.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0005129157.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0005261375.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0005285696.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0005550441.mp3


D:\MyApps\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
D:\MyApps\anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0005608787.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0005656534.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0005695311.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0005798527.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0005850999.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0005884980.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0005987148.mp3
Extracted feature from file: D:/PhD Program/Final Research/DAT

Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0011922080.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0011922905.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0011938737.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0012027536.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0012112600.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0012151268.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0012222183.mp3
Extracted feature from file: D:/PhD Program/Final Research/DAT

Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0030442334.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0030481897.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0030494794.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0030495725.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0031495046.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0031807708.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0031826774.mp3
Extracted feature from file: D:/PhD Program/Final Research/DAT

In [2]:
# Extract Features from data set - DS 1: MER Audio Traffic dataset: split 1 mp3 file into multiple observations
# one observation = 5 second duration
# - without Hilbert Transform
import os
os.chdir("D:/PhD Program/Final Research/OLR")

import pandas as pd
import Feature_Extraction_Util_V2 as util
import os

num_mfcc = 20
num_mel = 64
mel_cutoff_val = 0.01
hilbert_transform_val = "N"
MFCC_flag = "Y" 
feature_file_name = "DS_1_Feature"

if MFCC_flag == "Y":
    feature_file_name = feature_file_name + "_MFCC"
if hilbert_transform_val == "Y":
    feature_file_name = feature_file_name + "_hilbert_trans_ms_5sec"
else:
    feature_file_name = feature_file_name + "_no_hilbert_trans_ms_5sec"

feature_file_name = feature_file_name + ".csv"
print(feature_file_name)

feat_util = util.Feature_Util()
root_path = "D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset/"
class_names = ["Q1","Q2","Q3","Q4"]
#class_names = ["Q1"]
file_cntr = 1
for class_name in class_names:
    data_path = root_path+"/"+class_name
    for root, dirs, files in os.walk(data_path):
        temp_cntr = 0
        for file_name in files:
            #print(file_name)
            #print("Temp counter: " + str(temp_cntr))
            file_path = data_path +"/" +file_name
            db_row = {'sound_file_name': [file_name],
                      'sound_file_class': [class_name]}
            # read sound file
            ret_val, msg_val, full_signal_val, sr = feat_util.readSignal(file_path,hilbert_transform=hilbert_transform_val)
            
            sound_file_size = len(full_signal_val)/sr
            # 5 second window
            window_size_duration = 5*sr 

            # start after 3 seconds
            start_pos = 3*sr 
            slider_width = window_size_duration
            overlap_sample_size = sr
            end_pos = start_pos +  slider_width - overlap_sample_size
            exit_flag = False
            sample_cntr = 1
#             print(len(full_signal_val))
#             print(sr)
            while not exit_flag:
                end_pos = start_pos +  slider_width - overlap_sample_size
                if end_pos > len(full_signal_val):
                    end_pos = len(full_signal_val)
                    exit_flag = True    
                
                # Extract a sample from the whole signal 
                signal_val = full_signal_val[start_pos:end_pos]
#                 print(" Start: " + str(start_pos)+" , End: "+str(end_pos))
                start_pos = end_pos
                # get features and aggregations
                # F0
                ret_val, msg_str, feature_val =  feat_util.getFZero(signal_val)
                agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="F0",mean_agg = "Y", median_agg = "Y", max_agg = "Y", min_agg = "Y", IQR_agg = "N", sd_agg ="N")
                db_row = db_row | agg_val

                # RMS
                ret_val,msg_str, feature_val =  feat_util.getRMS(signal_val)
                agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="RMS",mean_agg = "Y", median_agg = "Y", max_agg = "Y", min_agg = "N", IQR_agg = "N", sd_agg ="N")
                db_row = db_row | agg_val

                # Spectral Centroid
                ret_val,msg_str, feature_val =  feat_util.getSpectralCentroid(signal_val)
                agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="spect_centroid")
                db_row = db_row | agg_val

                # Spectral Roll Off
                ret_val,msg_str, feature_val =  feat_util.getSpectralRolloff(signal_val)
                agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="spect_rolloff")
                db_row = db_row | agg_val

                # Spectral Flatness
                ret_val,msg_str, feature_val =  feat_util.getSpectralFlatness(signal_val)
                agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="spect_flat")
                db_row = db_row | agg_val

                # Spectral BW
                ret_val,msg_str, feature_val =  feat_util.getSpectralBW(signal_val)
                agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="spect_bw")
                db_row = db_row | agg_val

                # Spectral Contrast
                ret_val,msg_str, feature_val =  feat_util.getSpectralContrast(signal_val)
                agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="spect_contrast")
                db_row = db_row | agg_val

                # ZCR
                ret_val,msg_str, feature_val =  feat_util.getZCR(signal_val)
                agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="ZCR")
                db_row = db_row | agg_val

                # Beat / Tempo
                ret_val,msg_str, feature_val =  feat_util.getTempo(signal_val, sr)
                agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="tempo")
                db_row = db_row | agg_val

                # PLP
                ret_val,msg_str, feature_val =  feat_util.getPLP(signal_val, sr)
                agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="plp")
                db_row = db_row | agg_val

                # Power
                ret_val,msg_str, feature_val =  feat_util.getPower(signal_val, sr=sr)
                agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="power")
                db_row = db_row | agg_val
            
                # get MFCC's
                if MFCC_flag == "Y":
                    for mfcc_count in range(num_mfcc):
                        mfcc_str = "MFCC_" + str(mfcc_count)
                        ret_val, msg_str, feature_val = feat_util.getMFCC(signal_val, sr=sr, pos_val=mfcc_count)
                        agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name=mfcc_str)
                        db_row = db_row | agg_val

                # get MEL
                for mel_count in range(num_mel):
                    mel_str = "mel_" + str(mel_count)
                    ret_val, msg_str, feature_val = feat_util.getMel(signal_val, sr=sr, pos_val=mel_count, cutoff_val=mel_cutoff_val)
                    agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name=mel_str)
                    db_row = db_row | agg_val

                # get Loudness
                ret_val,msg_str, feature_val =  feat_util.getLoudness(file_path)
                #agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="loudness")
                db_row = db_row | {'loudness_mean':[feature_val]}

                # get Chromagram - To be included later. Hence commenting.
                # ret_val,msg_str, feature_val = feat_util.getChromagram(signal_val, sr=sr)
                # agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="chromagram")
                # db_row = db_row | agg_val

                temp_df = pd.DataFrame(db_row)
                # Build feature dataset
                if sample_cntr == 1:
                    sample_df = temp_df
                    #break
                else:
                    sample_df = pd.concat([sample_df,temp_df],axis=0)
                sample_cntr += 1
                
            data_df = sample_df

            # Build feature dataset
            if file_cntr == 1:
                final_df = data_df
                #break
            else:
                final_df = pd.concat([final_df,data_df],axis=0)
            file_cntr += 1
            print("Extracted feature from file: "+str(file_path))
#         break
#     break

# Save feature dataset
final_df = final_df.reset_index()
final_df = final_df.drop(columns = ['index'])
final_df.to_csv(feature_file_name, index=False)

DS_1_Feature_MFCC_no_hilbert_trans_ms_5sec.csv


D:\MyApps\anaconda3\lib\site-packages\librosa\core\spectrum.py:222: UserWarning: n_fft=384 is too small for input signal of length=345
  warnings.warn(
D:\MyApps\anaconda3\lib\site-packages\librosa\core\spectrum.py:222: UserWarning: n_fft=384 is too small for input signal of length=264
  warnings.warn(


Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0000040632.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0000082187.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0000249842.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0000364027.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0000392975.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0000414517.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0000442827.mp3
Extracted feature from file: D:/PhD Program/Final Research/DAT

D:\MyApps\anaconda3\lib\site-packages\librosa\core\spectrum.py:222: UserWarning: n_fft=384 is too small for input signal of length=267
  warnings.warn(


Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0000992846.mp3


...

Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0014134790.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0014475915.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0014489361.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0014507179.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0014584319.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0015026293.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0015653546.mp3
Extracted feature from file: D:/PhD Program/Final Research/DAT

D:\MyApps\anaconda3\lib\site-packages\librosa\core\spectrum.py:222: UserWarning: n_fft=384 is too small for input signal of length=98
  warnings.warn(


Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0028813196.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0029080694.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0029553110.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0029976595.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0030007733.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0030036616.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0030097602.mp3
Extracted feature from file: D:/PhD Program/Final Research/DAT

D:\MyApps\anaconda3\lib\site-packages\librosa\core\spectrum.py:222: UserWarning: n_fft=384 is too small for input signal of length=227
  warnings.warn(


Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0030369896.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0030442334.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0030481897.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0030494794.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0030495725.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0031495046.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0031807708.mp3
Extracted feature from file: D:/PhD Program/Final Research/DAT

In [3]:
# Extract Features from data set - DS 1: MER Audio Traffic dataset: split 1 mp3 file into multiple observations
# one observation = 5 second duration
# - with Hilbert Transform
import os
os.chdir("D:/PhD Program/Final Research/OLR")

import pandas as pd
import Feature_Extraction_Util_V2 as util
import os

num_mfcc = 20
num_mel = 64
mel_cutoff_val = 0.01
hilbert_transform_val = "Y"
MFCC_flag = "Y" 
feature_file_name = "DS_1_Feature"

if MFCC_flag == "Y":
    feature_file_name = feature_file_name + "_MFCC"
if hilbert_transform_val == "Y":
    feature_file_name = feature_file_name + "_hilbert_trans_ms_5sec"
else:
    feature_file_name = feature_file_name + "_no_hilbert_trans_ms_5sec"

feature_file_name = feature_file_name + ".csv"
print(feature_file_name)

feat_util = util.Feature_Util()
root_path = "D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset/"
class_names = ["Q1","Q2","Q3","Q4"]
#class_names = ["Q1"]
file_cntr = 1
for class_name in class_names:
    data_path = root_path+"/"+class_name
    for root, dirs, files in os.walk(data_path):
        temp_cntr = 0
        for file_name in files:
            #print(file_name)
            #print("Temp counter: " + str(temp_cntr))
            file_path = data_path +"/" +file_name
            db_row = {'sound_file_name': [file_name],
                      'sound_file_class': [class_name]}
            # read sound file
            ret_val, msg_val, full_signal_val, sr = feat_util.readSignal(file_path,hilbert_transform=hilbert_transform_val)
            
            sound_file_size = len(full_signal_val)/sr
            # 5 second window
            window_size_duration = 5*sr 

            # start after 3 seconds
            start_pos = 3*sr 
            slider_width = window_size_duration
            overlap_sample_size = sr
            end_pos = start_pos +  slider_width - overlap_sample_size
            exit_flag = False
            sample_cntr = 1
#             print(len(full_signal_val))
#             print(sr)
            while not exit_flag:
                end_pos = start_pos +  slider_width - overlap_sample_size
                if end_pos > len(full_signal_val):
                    end_pos = len(full_signal_val)
                    exit_flag = True    
                
                # Extract a sample from the whole signal 
                signal_val = full_signal_val[start_pos:end_pos]
#                 print(" Start: " + str(start_pos)+" , End: "+str(end_pos))
                start_pos = end_pos
                # get features and aggregations
                # F0
                ret_val, msg_str, feature_val =  feat_util.getFZero(signal_val)
                agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="F0",mean_agg = "Y", median_agg = "Y", max_agg = "Y", min_agg = "Y", IQR_agg = "N", sd_agg ="N")
                db_row = db_row | agg_val

                # RMS
                ret_val,msg_str, feature_val =  feat_util.getRMS(signal_val)
                agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="RMS",mean_agg = "Y", median_agg = "Y", max_agg = "Y", min_agg = "N", IQR_agg = "N", sd_agg ="N")
                db_row = db_row | agg_val

                # Spectral Centroid
                ret_val,msg_str, feature_val =  feat_util.getSpectralCentroid(signal_val)
                agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="spect_centroid")
                db_row = db_row | agg_val

                # Spectral Roll Off
                ret_val,msg_str, feature_val =  feat_util.getSpectralRolloff(signal_val)
                agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="spect_rolloff")
                db_row = db_row | agg_val

                # Spectral Flatness
                ret_val,msg_str, feature_val =  feat_util.getSpectralFlatness(signal_val)
                agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="spect_flat")
                db_row = db_row | agg_val

                # Spectral BW
                ret_val,msg_str, feature_val =  feat_util.getSpectralBW(signal_val)
                agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="spect_bw")
                db_row = db_row | agg_val

                # Spectral Contrast
                ret_val,msg_str, feature_val =  feat_util.getSpectralContrast(signal_val)
                agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="spect_contrast")
                db_row = db_row | agg_val

                # ZCR
                ret_val,msg_str, feature_val =  feat_util.getZCR(signal_val)
                agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="ZCR")
                db_row = db_row | agg_val

                # Beat / Tempo
                ret_val,msg_str, feature_val =  feat_util.getTempo(signal_val, sr)
                agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="tempo")
                db_row = db_row | agg_val

                # PLP
                ret_val,msg_str, feature_val =  feat_util.getPLP(signal_val, sr)
                agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="plp")
                db_row = db_row | agg_val

                # Power
                ret_val,msg_str, feature_val =  feat_util.getPower(signal_val, sr=sr)
                agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="power")
                db_row = db_row | agg_val
            
                # get MFCC's
                if MFCC_flag == "Y":
                    for mfcc_count in range(num_mfcc):
                        mfcc_str = "MFCC_" + str(mfcc_count)
                        ret_val, msg_str, feature_val = feat_util.getMFCC(signal_val, sr=sr, pos_val=mfcc_count)
                        agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name=mfcc_str)
                        db_row = db_row | agg_val

                # get MEL
                for mel_count in range(num_mel):
                    mel_str = "mel_" + str(mel_count)
                    ret_val, msg_str, feature_val = feat_util.getMel(signal_val, sr=sr, pos_val=mel_count, cutoff_val=mel_cutoff_val)
                    agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name=mel_str)
                    db_row = db_row | agg_val

                # get Loudness
                ret_val,msg_str, feature_val =  feat_util.getLoudness(file_path)
                #agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="loudness")
                db_row = db_row | {'loudness_mean':[feature_val]}

                # get Chromagram - To be included later. Hence commenting.
                # ret_val,msg_str, feature_val = feat_util.getChromagram(signal_val, sr=sr)
                # agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="chromagram")
                # db_row = db_row | agg_val

                temp_df = pd.DataFrame(db_row)
                # Build feature dataset
                if sample_cntr == 1:
                    sample_df = temp_df
                    #break
                else:
                    sample_df = pd.concat([sample_df,temp_df],axis=0)
                sample_cntr += 1
                
            data_df = sample_df

            # Build feature dataset
            if file_cntr == 1:
                final_df = data_df
                #break
            else:
                final_df = pd.concat([final_df,data_df],axis=0)
            file_cntr += 1
            print("Extracted feature from file: "+str(file_path))
#         break
#     break

# Save feature dataset
final_df = final_df.reset_index()
final_df = final_df.drop(columns = ['index'])
final_df.to_csv(feature_file_name, index=False)

DS_1_Feature_MFCC_hilbert_trans_ms_5sec.csv
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0000040632.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0000082187.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0000249842.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0000364027.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0000392975.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0000414517.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0000442827.mp3
Extracted feature 

Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0004032071.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0004037272.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0004122617.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0004141823.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0004339430.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0004428604.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0004447226.mp3
Extracted feature from file: D:/PhD Program/Final Research/DAT

Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0009904717.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0010085729.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0010344415.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0010392442.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0010465830.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0010482550.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0010487769.mp3
Extracted feature from file: D:/PhD Program/Final Research/DAT

Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0026158301.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0026727455.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0026753600.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0026753827.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0026795261.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0027002641.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0027035970.mp3
Extracted feature from file: D:/PhD Program/Final Research/DAT

Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q2/MT0001975228.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q2/MT0001993588.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q2/MT0002050671.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q2/MT0002372242.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q2/MT0002385077.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q2/MT0002399275.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q2/MT0002415184.mp3
Extracted feature from file: D:/PhD Program/Final Research/DAT

...

Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q3/MT0011969592.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q3/MT0012020062.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q3/MT0012292188.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q3/MT0012317309.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q3/MT0012634038.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q3/MT0012675695.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q3/MT0013044535.mp3
Extracted feature from file: D:/PhD Program/Final Research/DAT

Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0000336135.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0000348553.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0000369789.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0000460427.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0000522789.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0000550961.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0000636335.mp3
Extracted feature from file: D:/PhD Program/Final Research/DAT

Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0004911454.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0005074629.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0005091539.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0005129157.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0005261375.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0005285696.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0005550441.mp3
Extracted feature from file: D:/PhD Program/Final Research/DAT

Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0011376343.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0011380622.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0011702301.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0011862487.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0011886148.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0011895175.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0011906602.mp3
Extracted feature from file: D:/PhD Program/Final Research/DAT

Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0030007733.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0030036616.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0030097602.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0030282917.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0030286920.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0030319227.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0030369896.mp3
Extracted feature from file: D:/PhD Program/Final Research/DAT

In [4]:
# Extract Features from data set - DS 1: MER Audio Traffic dataset: split 1 mp3 file into multiple observations
# one observation = 1 second duration
# - without Hilbert Transform
import os
os.chdir("D:/PhD Program/Final Research/OLR")

import pandas as pd
import Feature_Extraction_Util_V2 as util
import os

num_mfcc = 20
num_mel = 64
mel_cutoff_val = 0.01
hilbert_transform_val = "N"
MFCC_flag = "Y" 
feature_file_name = "DS_1_Feature"

if MFCC_flag == "Y":
    feature_file_name = feature_file_name + "_MFCC"
if hilbert_transform_val == "Y":
    feature_file_name = feature_file_name + "_hilbert_trans_ms_1sec"
else:
    feature_file_name = feature_file_name + "_no_hilbert_trans_ms_1sec"

feature_file_name = feature_file_name + ".csv"
print(feature_file_name)

feat_util = util.Feature_Util()
root_path = "D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset/"
class_names = ["Q1","Q2","Q3","Q4"]
#class_names = ["Q1"]
file_cntr = 1
for class_name in class_names:
    data_path = root_path+"/"+class_name
    for root, dirs, files in os.walk(data_path):
        temp_cntr = 0
        for file_name in files:
            #print(file_name)
            #print("Temp counter: " + str(temp_cntr))
            file_path = data_path +"/" +file_name
            db_row = {'sound_file_name': [file_name],
                      'sound_file_class': [class_name]}
            # read sound file
            ret_val, msg_val, full_signal_val, sr = feat_util.readSignal(file_path,hilbert_transform=hilbert_transform_val)
            
            sound_file_size = len(full_signal_val)/sr
            # 5 second window
            window_size_duration = 1*sr 

            # start after 3 seconds
            start_pos = 3*sr 
            slider_width = window_size_duration
            overlap_sample_size = int(sr/4)
            end_pos = start_pos +  slider_width - overlap_sample_size
            exit_flag = False
            sample_cntr = 1
#             print(len(full_signal_val))
#             print(sr)
            while not exit_flag:
                end_pos = start_pos +  slider_width - overlap_sample_size
                if end_pos > len(full_signal_val):
                    end_pos = len(full_signal_val)
                    exit_flag = True    
                
                # Extract a sample from the whole signal 
                signal_val = full_signal_val[start_pos:end_pos]
#                 print(" Start: " + str(start_pos)+" , End: "+str(end_pos))
                start_pos = end_pos
                # get features and aggregations
                # F0
                ret_val, msg_str, feature_val =  feat_util.getFZero(signal_val)
                agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="F0",mean_agg = "Y", median_agg = "Y", max_agg = "Y", min_agg = "Y", IQR_agg = "N", sd_agg ="N")
                db_row = db_row | agg_val

                # RMS
                ret_val,msg_str, feature_val =  feat_util.getRMS(signal_val)
                agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="RMS",mean_agg = "Y", median_agg = "Y", max_agg = "Y", min_agg = "N", IQR_agg = "N", sd_agg ="N")
                db_row = db_row | agg_val

                # Spectral Centroid
                ret_val,msg_str, feature_val =  feat_util.getSpectralCentroid(signal_val)
                agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="spect_centroid")
                db_row = db_row | agg_val

                # Spectral Roll Off
                ret_val,msg_str, feature_val =  feat_util.getSpectralRolloff(signal_val)
                agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="spect_rolloff")
                db_row = db_row | agg_val

                # Spectral Flatness
                ret_val,msg_str, feature_val =  feat_util.getSpectralFlatness(signal_val)
                agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="spect_flat")
                db_row = db_row | agg_val

                # Spectral BW
                ret_val,msg_str, feature_val =  feat_util.getSpectralBW(signal_val)
                agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="spect_bw")
                db_row = db_row | agg_val

                # Spectral Contrast
                ret_val,msg_str, feature_val =  feat_util.getSpectralContrast(signal_val)
                agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="spect_contrast")
                db_row = db_row | agg_val

                # ZCR
                ret_val,msg_str, feature_val =  feat_util.getZCR(signal_val)
                agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="ZCR")
                db_row = db_row | agg_val

                # Beat / Tempo
                ret_val,msg_str, feature_val =  feat_util.getTempo(signal_val, sr)
                agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="tempo")
                db_row = db_row | agg_val

                # PLP
                ret_val,msg_str, feature_val =  feat_util.getPLP(signal_val, sr)
                agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="plp")
                db_row = db_row | agg_val

                # Power
                ret_val,msg_str, feature_val =  feat_util.getPower(signal_val, sr=sr)
                agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="power")
                db_row = db_row | agg_val
            
                # get MFCC's
                if MFCC_flag == "Y":
                    for mfcc_count in range(num_mfcc):
                        mfcc_str = "MFCC_" + str(mfcc_count)
                        ret_val, msg_str, feature_val = feat_util.getMFCC(signal_val, sr=sr, pos_val=mfcc_count)
                        agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name=mfcc_str)
                        db_row = db_row | agg_val

                # get MEL
                for mel_count in range(num_mel):
                    mel_str = "mel_" + str(mel_count)
                    ret_val, msg_str, feature_val = feat_util.getMel(signal_val, sr=sr, pos_val=mel_count, cutoff_val=mel_cutoff_val)
                    agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name=mel_str)
                    db_row = db_row | agg_val

                # get Loudness
                ret_val,msg_str, feature_val =  feat_util.getLoudness(file_path)
                #agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="loudness")
                db_row = db_row | {'loudness_mean':[feature_val]}

                # get Chromagram - To be included later. Hence commenting.
                # ret_val,msg_str, feature_val = feat_util.getChromagram(signal_val, sr=sr)
                # agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="chromagram")
                # db_row = db_row | agg_val

                temp_df = pd.DataFrame(db_row)
                # Build feature dataset
                if sample_cntr == 1:
                    sample_df = temp_df
                    #break
                else:
                    sample_df = pd.concat([sample_df,temp_df],axis=0)
                sample_cntr += 1
                
            data_df = sample_df

            # Build feature dataset
            if file_cntr == 1:
                final_df = data_df
                #break
            else:
                final_df = pd.concat([final_df,data_df],axis=0)
            file_cntr += 1
            print("Extracted feature from file: "+str(file_path))
#         break
#     break

# Save feature dataset
final_df = final_df.reset_index()
final_df = final_df.drop(columns = ['index'])
final_df.to_csv(feature_file_name, index=False)

DS_1_Feature_MFCC_no_hilbert_trans_ms_1sec.csv


D:\MyApps\anaconda3\lib\site-packages\librosa\core\spectrum.py:222: UserWarning: n_fft=384 is too small for input signal of length=65
  warnings.warn(


Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0000040632.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0000082187.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0000249842.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0000364027.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0000392975.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0000414517.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0000442827.mp3
Extracted feature from file: D:/PhD Program/Final Research/DAT

D:\MyApps\anaconda3\lib\site-packages\librosa\core\spectrum.py:222: UserWarning: n_fft=384 is too small for input signal of length=9
  warnings.warn(


Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0000992846.mp3


...

Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0014134790.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0014475915.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0014489361.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0014507179.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0014584319.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0015026293.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0015653546.mp3
Extracted feature from file: D:/PhD Program/Final Research/DAT

D:\MyApps\anaconda3\lib\site-packages\librosa\core\spectrum.py:222: UserWarning: n_fft=384 is too small for input signal of length=34
  warnings.warn(


Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0028813196.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0029080694.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0029553110.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0029976595.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0030007733.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0030036616.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0030097602.mp3
Extracted feature from file: D:/PhD Program/Final Research/DAT

D:\MyApps\anaconda3\lib\site-packages\librosa\core\spectrum.py:222: UserWarning: n_fft=384 is too small for input signal of length=33
  warnings.warn(


Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0030369896.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0030442334.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0030481897.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0030494794.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0030495725.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0031495046.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0031807708.mp3
Extracted feature from file: D:/PhD Program/Final Research/DAT

In [1]:
# Extract Features from data set - DS 1: MER Audio Traffic dataset: split 1 mp3 file into multiple observations
# one observation = 1 second duration
# - with Hilbert Transform
import os
os.chdir("D:/PhD Program/Final Research/OLR")

import pandas as pd
import Feature_Extraction_Util_V2 as util
import os

num_mfcc = 20
num_mel = 64
mel_cutoff_val = 0.01
hilbert_transform_val = "Y"
MFCC_flag = "Y" 
feature_file_name = "DS_1_Feature"

if MFCC_flag == "Y":
    feature_file_name = feature_file_name + "_MFCC"
if hilbert_transform_val == "Y":
    feature_file_name = feature_file_name + "_hilbert_trans_ms_1sec"
else:
    feature_file_name = feature_file_name + "_no_hilbert_trans_ms_1sec"

feature_file_name = feature_file_name + ".csv"
print(feature_file_name)

feat_util = util.Feature_Util()
root_path = "D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset/"
class_names = ["Q1","Q2","Q3","Q4"]
#class_names = ["Q1"]
file_cntr = 1
for class_name in class_names:
    data_path = root_path+"/"+class_name
    for root, dirs, files in os.walk(data_path):
        temp_cntr = 0
        for file_name in files:
            #print(file_name)
            #print("Temp counter: " + str(temp_cntr))
            file_path = data_path +"/" +file_name
            db_row = {'sound_file_name': [file_name],
                      'sound_file_class': [class_name]}
            # read sound file
            ret_val, msg_val, full_signal_val, sr = feat_util.readSignal(file_path,hilbert_transform=hilbert_transform_val)
            
            sound_file_size = len(full_signal_val)/sr
            # 5 second window
            window_size_duration = 1*sr 

            # start after 3 seconds
            start_pos = 3*sr 
            slider_width = window_size_duration
            overlap_sample_size = int(sr/4)
            end_pos = start_pos +  slider_width - overlap_sample_size
            exit_flag = False
            sample_cntr = 1
#             print(len(full_signal_val))
#             print(sr)
            while not exit_flag:
                end_pos = start_pos +  slider_width - overlap_sample_size
                if end_pos > len(full_signal_val):
                    end_pos = len(full_signal_val)
                    exit_flag = True    
                
                # Extract a sample from the whole signal 
                signal_val = full_signal_val[start_pos:end_pos]
#                 print(" Start: " + str(start_pos)+" , End: "+str(end_pos))
                start_pos = end_pos
                # get features and aggregations
                # F0
                ret_val, msg_str, feature_val =  feat_util.getFZero(signal_val)
                agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="F0",mean_agg = "Y", median_agg = "Y", max_agg = "Y", min_agg = "Y", IQR_agg = "N", sd_agg ="N")
                db_row = db_row | agg_val

                # RMS
                ret_val,msg_str, feature_val =  feat_util.getRMS(signal_val)
                agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="RMS",mean_agg = "Y", median_agg = "Y", max_agg = "Y", min_agg = "N", IQR_agg = "N", sd_agg ="N")
                db_row = db_row | agg_val

                # Spectral Centroid
                ret_val,msg_str, feature_val =  feat_util.getSpectralCentroid(signal_val)
                agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="spect_centroid")
                db_row = db_row | agg_val

                # Spectral Roll Off
                ret_val,msg_str, feature_val =  feat_util.getSpectralRolloff(signal_val)
                agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="spect_rolloff")
                db_row = db_row | agg_val

                # Spectral Flatness
                ret_val,msg_str, feature_val =  feat_util.getSpectralFlatness(signal_val)
                agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="spect_flat")
                db_row = db_row | agg_val

                # Spectral BW
                ret_val,msg_str, feature_val =  feat_util.getSpectralBW(signal_val)
                agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="spect_bw")
                db_row = db_row | agg_val

                # Spectral Contrast
                ret_val,msg_str, feature_val =  feat_util.getSpectralContrast(signal_val)
                agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="spect_contrast")
                db_row = db_row | agg_val

                # ZCR
                ret_val,msg_str, feature_val =  feat_util.getZCR(signal_val)
                agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="ZCR")
                db_row = db_row | agg_val

                # Beat / Tempo
                ret_val,msg_str, feature_val =  feat_util.getTempo(signal_val, sr)
                agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="tempo")
                db_row = db_row | agg_val

                # PLP
                ret_val,msg_str, feature_val =  feat_util.getPLP(signal_val, sr)
                agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="plp")
                db_row = db_row | agg_val

                # Power
                ret_val,msg_str, feature_val =  feat_util.getPower(signal_val, sr=sr)
                agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="power")
                db_row = db_row | agg_val
            
                # get MFCC's
                if MFCC_flag == "Y":
                    for mfcc_count in range(num_mfcc):
                        mfcc_str = "MFCC_" + str(mfcc_count)
                        ret_val, msg_str, feature_val = feat_util.getMFCC(signal_val, sr=sr, pos_val=mfcc_count)
                        agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name=mfcc_str)
                        db_row = db_row | agg_val

                # get MEL
                for mel_count in range(num_mel):
                    mel_str = "mel_" + str(mel_count)
                    ret_val, msg_str, feature_val = feat_util.getMel(signal_val, sr=sr, pos_val=mel_count, cutoff_val=mel_cutoff_val)
                    agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name=mel_str)
                    db_row = db_row | agg_val

                # get Loudness
                ret_val,msg_str, feature_val =  feat_util.getLoudness(file_path)
                #agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="loudness")
                db_row = db_row | {'loudness_mean':[feature_val]}

                # get Chromagram - To be included later. Hence commenting.
                # ret_val,msg_str, feature_val = feat_util.getChromagram(signal_val, sr=sr)
                # agg_val, msg_str = feat_util.calcAggregations(feature_val, feature_name="chromagram")
                # db_row = db_row | agg_val

                temp_df = pd.DataFrame(db_row)
                # Build feature dataset
                if sample_cntr == 1:
                    sample_df = temp_df
                    #break
                else:
                    sample_df = pd.concat([sample_df,temp_df],axis=0)
                sample_cntr += 1
                
            data_df = sample_df

            # Build feature dataset
            if file_cntr == 1:
                final_df = data_df
                #break
            else:
                final_df = pd.concat([final_df,data_df],axis=0)
            file_cntr += 1
            print("Extracted feature from file: "+str(file_path))
#         break
#     break

# Save feature dataset
final_df = final_df.reset_index()
final_df = final_df.drop(columns = ['index'])
final_df.to_csv(feature_file_name, index=False)

DS_1_Feature_MFCC_hilbert_trans_ms_1sec.csv


D:\MyApps\anaconda3\lib\site-packages\librosa\core\audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
D:\MyApps\anaconda3\lib\site-packages\librosa\core\spectrum.py:222: UserWarning: n_fft=384 is too small for input signal of length=65
  warnings.warn(
D:\MyApps\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
D:\MyApps\anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
D:\MyApps\anaconda3\lib\site-packages\librosa\core\spectrum.py:222: UserWarning: n_fft=384 is too small for input signal of length=6
  warnings.warn(


Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0000040632.mp3


D:\MyApps\anaconda3\lib\site-packages\librosa\core\audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0000082187.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0000249842.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0000364027.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0000392975.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0000414517.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0000442827.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q1/MT0000540286.mp3
Extracted feature from file: D:/PhD Program/Final Research/DAT

...

Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0014134790.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0014475915.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0014489361.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0014507179.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0014584319.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0015026293.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0015653546.mp3
Extracted feature from file: D:/PhD Program/Final Research/DAT

D:\MyApps\anaconda3\lib\site-packages\librosa\core\spectrum.py:222: UserWarning: n_fft=384 is too small for input signal of length=34
  warnings.warn(


Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0028813196.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0029080694.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0029553110.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0029976595.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0030007733.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0030036616.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0030097602.mp3
Extracted feature from file: D:/PhD Program/Final Research/DAT

D:\MyApps\anaconda3\lib\site-packages\librosa\core\spectrum.py:222: UserWarning: n_fft=384 is too small for input signal of length=33
  warnings.warn(


Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0030369896.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0030442334.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0030481897.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0030494794.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0030495725.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0031495046.mp3
Extracted feature from file: D:/PhD Program/Final Research/DATASETS - Final Paper/DS - 1 MER_audio_taffc_dataset//Q4/MT0031807708.mp3
Extracted feature from file: D:/PhD Program/Final Research/DAT